In [2]:
import mxnet as mx
import numpy as np
from mxnet import nd, autograd
from mxnet import gluon
mx.random.seed(1)

ctx = mx.cpu()

In [4]:
batch_size = 64
def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)
train_data = gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)

In [5]:
W1 = nd.random_normal(shape=(784, 256), ctx=ctx) * .01
b1 = nd.random_normal(shape=256, ctx=ctx) * .01

W2 = nd.random_normal(shape=(256,128), ctx=ctx) *.01
b2 = nd.random_normal(shape=128, ctx=ctx) * .01

W3 = nd.random_normal(shape=(128,10), ctx=ctx) *.01
b3 = nd.random_normal(shape=10, ctx=ctx) *.01

params = [W1, b1, W2, b2, W3, b3]

In [6]:
for param in params:
    param.attach_grad()

In [7]:
def relu(X):
    return nd.maximum(X, 0)

In [8]:
def dropout(X, drop_probability):
    keep_probability = 1 - drop_probability
    mask = nd.random_uniform(0, 1.0, X.shape, ctx=X.context) < keep_probability
    
    if keep_probability > 0.0:
        scale = (1 / keep_probability)
    else:
        scale = 0.0
    return mask * X * scale

In [9]:
A = nd.arange(20).reshape((5, 4))
dropout(A, 0.0)


[[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]
 [ 12.  13.  14.  15.]
 [ 16.  17.  18.  19.]]
<NDArray 5x4 @cpu(0)>

In [10]:
dropout(A, 0.5)


[[  0.   0.   0.   6.]
 [  8.  10.   0.   0.]
 [  0.  18.  20.  22.]
 [  0.  26.   0.   0.]
 [ 32.  34.  36.   0.]]
<NDArray 5x4 @cpu(0)>

In [11]:
dropout(A, 1.0)


[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
<NDArray 5x4 @cpu(0)>

In [12]:
def softmax(y_linear):
    exp = nd.exp(y_linear-nd.max(y_linear))
    partition = nd.nansum(exp, axis=0, exclude=True).reshape((-1,1))
    return exp / partition

In [13]:
def softmax_cross_entropy(yhat_linear, y):
    return - nd.nansum(y * nd.log_softmax(yhat_linear), axis=0, exclude=True)

In [14]:
def net(X, drop_prob=0.0):
    h1_linear = nd.dot(X, W1) + b1
    h1 = relu(h1_linear)
    h1 = dropout(h1, drop_prob)
    
    h2_linear = nd.dot(h1, W2) + b2
    h2 = relu(h2_linear)
    h2 = dropout(h2, drop_prob)
    
    yhat_linear = nd.dot(h2, W3) + b3
    return yhat_linear

In [15]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [16]:
def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx).reshape((-1,784))
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        numerator += nd.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator / denominator).asscalar()

In [ ]:
epochs = 10
moving_loss = 0.
learning_rate = .001

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx).reshape((-1,784))
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 10)
        with autograd.record():
            ################################
            #   Drop out 50% of hidden activations on the forward pass
            ################################
            output = net(data, drop_prob=.5)
            loss = softmax_cross_entropy(output, label_one_hot)
        loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if i == 0:
            moving_loss = nd.mean(loss).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(loss).asscalar()

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))

Epoch 0. Loss: 0.762779053355, Train_acc 0.850483, Test_acc 0.8554
Epoch 1. Loss: 0.383686805373, Train_acc 0.920917, Test_acc 0.9226
Epoch 2. Loss: 0.300015770341, Train_acc 0.94505, Test_acc 0.9454
Epoch 3. Loss: 0.246679364224, Train_acc 0.957517, Test_acc 0.954
Epoch 4. Loss: 0.220242559198, Train_acc 0.96405, Test_acc 0.9605
Epoch 5. Loss: 0.196379120941, Train_acc 0.970567, Test_acc 0.9657
Epoch 6. Loss: 0.176997496793, Train_acc 0.973, Test_acc 0.9698
Epoch 7. Loss: 0.169468515943, Train_acc 0.976167, Test_acc 0.9698
